# Regression model with Keras

### Part D. Building a baseline model with normalized data and increased number of epochs

Importing necessary libraries and loading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

In [4]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [5]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [6]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

##### Now we normalize data

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Installing tenzor flow and Keras

In [8]:
pip install keras-tuner --upgrade

Requirement already up-to-date: keras-tuner in c:\users\sdovl\anaconda3\lib\site-packages (1.0.3)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied, skipping upgrade: decorator in c:\users\sdovl\anaconda3\lib\site-packages (from ipython->keras-tuner) (4.4.2)



In [9]:
import keras_tuner as kt
from tensorflow import keras

Creating regression model

In [10]:
from keras.models import Sequential
from keras.layers import Dense

In [11]:
n_cols = predictors_norm.shape[1] 

In [12]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [13]:
# build the model
model = regression_model()

###### 1. splitting data on train and test

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

###### 2.training model on train data

In [15]:
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
23/23 [==============================] - 17s 1ms/step - loss: 1640.0948
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 1577.9015
Epoch 3/50
23/23 [==============================] - 0s 1ms/step - loss: 1635.7370
Epoch 4/50
23/23 [==============================] - 0s 1ms/step - loss: 1558.5397
Epoch 5/50
23/23 [==============================] - 0s 1ms/step - loss: 1518.4436
Epoch 6/50
23/23 [==============================] - ETA: 0s - loss: 1718.79 - 0s 1ms/step - loss: 1502.5056
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 1386.0918
Epoch 8/50
23/23 [==============================] - 0s 1ms/step - loss: 1315.1410
Epoch 9/50
23/23 [==============================] - 0s 1ms/step - loss: 1176.9913
Epoch 10/50
23/23 [==============================] - 0s 2ms/step - loss: 1049.4138
Epoch 11/50
23/23 [==============================] - 0s 1ms/step - loss: 833.8385
Epoch 12/50
23/23 [==============================] - 0s 1ms/step - los

###### 3. Evaluating model on test data

In [16]:
loss_val = model.evaluate(X_test, y_test)
loss_val

10/10 [==============================] - 0s 1ms/step - loss: 134.0753


134.0752716064453

In [17]:
y_pred = model.predict(X_test)

###### computing mse between the predicted and actual values

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
mse = mean_squared_error(y_test, y_pred)
mse

134.07527286914868

###### 4. Repeating steps 1-3 50 times and create list of 50 mse

In [20]:
number_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, number_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    loss_val = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mse)
    print(" mse "+str(i+1)+": "+str(mse))


 mse 1: 100.7394983223837
 mse 2: 97.73592240878953
 mse 3: 45.65158743830114
 mse 4: 42.12250585522001
 mse 5: 38.11944816968029
 mse 6: 41.176909488401996
 mse 7: 41.93495730278365
 mse 8: 31.83189335964137
 mse 9: 33.07006955811222
 mse 10: 29.526437688920232
 mse 11: 30.425693895821023
 mse 12: 24.314475225243395
 mse 13: 34.52260137897413
 mse 14: 33.379287390758925
 mse 15: 27.041364125267865
 mse 16: 24.14693852465208
 mse 17: 26.508756226143888
 mse 18: 26.877542874600262
 mse 19: 25.281275860617935
 mse 20: 27.10115568739056
 mse 21: 24.483613871200546
 mse 22: 24.2559595225129
 mse 23: 20.663637091270697
 mse 24: 25.92449849171597
 mse 25: 24.659068430095246
 mse 26: 28.2335657371885
 mse 27: 22.95969790655577
 mse 28: 23.710013540578398
 mse 29: 26.35198340598571
 mse 30: 23.481143143244015
 mse 31: 21.201401685897277
 mse 32: 20.39602622556463
 mse 33: 23.172843482403362
 mse 34: 21.212553702053555
 mse 35: 23.021759231841845
 mse 36: 28.845484300223088
 mse 37: 19.34203130

###### 5.Finding mean and variance of MSE

In [21]:
mse = np.array(mse)
mean = np.mean(mse)
standard_deviation = np.std(mse)

print('\n')
print("The mean and standard deviation of " +str(number_mean_squared_errors) + " mean squared errors of normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))



The mean and standard deviation of 50 mean squared errors of normalized data. Total number of epochs for each training is: 50

Mean: 19.665361127043028
Standard Deviation: 0.0


## Now we can see that mean of mean squred errors is significally reduced comparing with part B when we added hidden layers to our model and it's best model in comparison to other models.